CO2 Emissions Prediction
Objetivo
Extrair indicadores do World Development Indicators (WDI) via API do Banco Mundial e gerar:
data/raw/wdi_long.csv (formato longo: país–indicador–ano–valor)
data/processed/wdi_wide.csv (formato wide estilo Kaggle: 1 linha por país+indicador, colunas por ano)

In [2]:
import logging
from pathlib import Path

import requests
import pandas as pd

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
logger = logging.getLogger("wdi_etl")

Configurações do ETL
INDICATORS: lista dos códigos WDI (fonte oficial)
YEARS: intervalo 2000–2020
PATH_LONG/PATH_WIDE: saídas em disco

In [3]:
INDICATORS = [
    "EG.ELC.ACCS.ZS",       # Access to electricity (% of population)
    "AG.LND.AGRI.ZS",       # Agricultural land (% of land area)
    "ER.H2O.FWVT.ZS",       # Annual freshwater withdrawals, total (% of internal resources)
    "AG.LND.ARBL.ZS",       # Arable land (% of land area)
    "AG.LND.FRST.ZS",       # Forest area (% of land area)
    "EG.USE.ELEC.KH.PC",    # Electric power consumption (kWh per capita)
    "EG.USE.PCAP.KG.OE",    # Energy use (kg of oil equivalent per capita)
    "EG.ELC.RNEW.ZS",       # Renewable electricity output (% of total electricity output)
    "EG.FEC.RNEW.ZS",       # Renewable energy consumption (% of total final energy consumption)
    "SP.POP.GROW",          # Population growth (annual %)
    "NY.GDP.PCAP.CD",       # GDP per capita (current US$)
    "EN.ATM.CO2E.PC",       # CO2 emissions (metric tons per capita)
]

YEARS = list(range(2000, 2021))
DATE_RANGE = f"{min(YEARS)}:{max(YEARS)}"

BASE = "https://api.worldbank.org/v2"

PATH_LONG = Path("data/raw/wdi_long.csv")
PATH_WIDE = Path("data/processed/wdi_wide.csv")

In [4]:
Path("data/raw").mkdir(parents=True, exist_ok=True)
Path("data/processed").mkdir(parents=True, exist_ok=True)
logger.info("Pastas data/raw e data/processed prontas.")

2026-02-24 22:56:09,629 | INFO | Pastas data/raw e data/processed prontas.


Função utilitária: paginação da APIA API do World Bank é paginada. A função abaixo:
percorre todas as páginas
acumula os itens
retorna uma lista única

In [5]:
def wb_get_all_pages(url: str, params: dict) -> list:
    """
    Busca todas as páginas de um endpoint da API v2 do World Bank.

    Estrutura típica de resposta:
      data[0] -> metadados (páginas, total, etc.)
      data[1] -> lista de registros

    Retorno:
      Lista com todos os itens agregados de todas as páginas.
    """
    params = dict(params)
    params.setdefault("format", "json")
    params.setdefault("per_page", 20000)  # tenta minimizar paginação

    page = 1
    out = []

    while True:
        params["page"] = page
        resp = requests.get(url, params=params, timeout=60)
        resp.raise_for_status()
        data = resp.json()

        # Quando não há dados: data[1] vem None
        if not isinstance(data, list) or len(data) < 2 or data[1] is None:
            break

        meta, items = data[0], data[1]
        out.extend(items)

        pages = int(meta.get("pages", 1))
        if page >= pages:
            break

        page += 1

    return out

Países e filtro de agregadosBaixamos o catálogo de países para:
identificar e remover “Aggregates” (World, regiões, etc.)
manter apenas países/territórios como unidades de análise